### 3. Механизм внимания

В предыдущей главе мы готовили человеческий текст к восприятию машиной.  
Теперь рассмотрим основной механизм больших языковых моделей, разбив рассмотрение на 4 усложняющихся шага.

![](images/llm3.2.png)

![](images/llm3.3.png)

### 3.1 Проблема длины контекста (шаг 0)

Что такое перевод? - Это сопоставления токенов из разных словарей (с позиции этой книги).

А т.к. у различных языков различные грамматики, то минимальный смысловой токен - предложение (перевод слово - к слову невозможен, рисунок).   
Т.е. необходим учет контекста, причем чем он длиннее - тем лучше, но его сложнее вычислить - проблема.

Решением этой проблемы были рекурентные нейронные сети (РНН). В этих сетях результат работы одного блока, идет на вход следующего, откуда и берется название (рекурентный член последовательности вычисляется на основе предыдущего): 
1. Входной текст подается в шифратор 
2. Шифратор обновляет значения слоев (состояния скрытых слоев, hidden states), так, что в последнем слое зашифрован контекст всего входящего текста.
3. Дешифратор получает этот последний слой, чтобы генерировать перевод предложения (уже не рекурентно, опираясь каждый раз на слой-контекст, который не меняется).

Т.е. нейронная сеть, построенная на рекурентном механизме способна работать с контекстом как векторным представлением, но без динамики - дешифратору доступно только последнее состояние контекста. Таким образом, по построению, возникает ограничение на длину и сложность входного текста. 

![](images/llm3.4.png)

![](images/llm3.5.png)

### 3.2. Идея внимания

В 2014 году был предложен механизм, позволяющий дешифратору получать все предыдущие состояния, а не только последнее. Причем дешифратор может ранжировать вклад этих состояний при генерации выходного токена. Ранжирование происходит через вычисление весовых коэфиициентов. После чего с помощью весовых коэффициентов вычисляется векторное представление контекста. Это представление служит для измерения контекста. А т.к. следующий токен предсказывается на основе контекста, можно оценить корректность этого измерения (и обновить это измерение на шаге обучения). 

В общем, идея внимания - это идея построения не просто понятного машине контекста(какое-то векторное представление), а эффективное с точки зрения предсказаний (такое векторное представление, которое действительно отражает смысловые связи).

![](images/llm3.6.png)

### 3.3 Вычисление внимания
#### 3.3.1. Вычисление внимания относительно одной позиции.  

На этом шаге показавыается способ измерения важности токенов - вычисление весов внимания (attention weight). Обратим внимание, что эти веса вычисляются относительно позиции. 

Есть входящая последовательность токенов.  
Каждый элемент последовательности, представлен как 3-мерный вектор (размерность 3 выбрана для простоты иллюстраций).

Наша цель — вычислить векторы контекста относительно каждого элемента входящей последовательности.  
Для этого: 
- Вычислим промежуточные значения помощью скалярного произведения. Эти промежуточные значения называются, называются оценками внимания (attention score).
- Нормализуем эти значения. Нормализованные оценки внимания называются весами (attention weight).

Скалярное произведение определим как сумму поэлементного умножения. 
"Физический смысл" скалярного произведения - определение степени "перекрытия" векторов. Так, ортогональные вектора (1, 0) и (0, 1) вообще не "перекрываются" и дадут 0. Т.е. чем больше значение скалярное произведение - тем больше их "перекрытие" и оценка внимания. 

![](images/llm3.8.png)

![](images/llm3.9.png) 

![](images/llm3.10.png)

![](images/llm3.11.png) 

![](images/llm3.13.png) 

In [1]:
# Векторное представление входящий последовательности.

import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

# Вычисление оценок внимания относительно одного вектора (одной позиции) входящей матрицы.

query = inputs[1]
attn_score = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_score[i] = torch.dot(x_i, query)
print("Оценки внимания:", attn_score)

# Нормализация 
# Использование softmax предпочтительно, т.к. гарантирует, что веса всегда будут положительными. 
# Это позволяет интерпретировать выходные данные как вероятности или относительную важность 
# Показана упрощенная реализация функции softmax, далее используется стандартная. 

def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
 
attn_weights_norm_2 = softmax_naive(attn_score)
print("Веса внимания:", attn_weights_norm_2)
print("Тест нормализации:", attn_weights_norm_2.sum())

# Вычисление векторного представления контекста относительно одной(inputs[1]) позиции 

query = inputs[1] 
context_vec = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec += attn_weights_norm_2[i]*x_i
print('Вектор контекста: ', context_vec)

Оценки внимания: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Веса внимания: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Тест нормализации: tensor(1.)
Вектор контекста:  tensor([0.4419, 0.6515, 0.5683])


#### 3.3.2. Вычисление внимания относительно всех позиций.  

Вычислим матрицу весов внимания для всей входящей матрицы. Желтым выделен посчитанный ранее вектор:  

![](images/llm3.12.png) 



In [2]:
# Вычисление оценок внимания с помощью скалярного произведения

attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print('Оценки внимания:\n', attn_scores)

# Код выше можно записать с помощью матричного умножения
# тензора inputs и транспонированного тензора inputs.T
# далее используется матричная запись умножения

"""attn_scores = inputs @ inputs.T"""

# Нормализация 

attn_weights = torch.softmax(attn_scores, dim=1)
print('Веса внимания:\n',attn_weights)
print('Проверка нормализации:\n',[row.sum() for row in attn_weights])

# Вычисление контекста

all_context_vecs = attn_weights @ inputs
print('Матрица векторных представлений контекста:\n', all_context_vecs) 

Оценки внимания:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
Веса внимания:
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
Проверка нормализации:
 [tensor(1.0000), tensor(1.), tensor(1.0000), tensor(1.), tensor(1.), tensor(1.)]
Матрица векторных представлений контекста:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 

#### 3.3.3 Механизм улучшения измерения внимания (trainable attention weights)

Мы научились вычислять векторное представление контекста, относительно позиции - считаем, что контекст совпадает, если векторные представления "накладываются" при скалярном произведении.   

Но как расширить "глубину" контекста от предложений и абзацев до глав и книг? Необходимо усложнить механизм внимания, как-то учесть значимость контекста не только конкретного входного текста, но и всех возможных входных текстов. А т.к. "все возможные" тексты обработать нельзя, искомый механизм должен быть механизмом памяти, т.е. изменения от входного текста. Для чего вводятся изменяемые объекты - параметры, условно (!) называемые "запрос", "ключ", "значение". Эти называния отсылают к языку баз данных, подразумевая что для входящего токена-запроса высчитывается пара ключ - значение, соотносящая запрос и контекст. Необходимо иметь в виду, что это - метафора, скрывающая скалярное умножение входящего вектора и вектора, хранящегося (и изменяющегося) в памяти. Необходимо также иметь в виду фундаментальную разницу между вычислением контекста на основе только входящих данных (РНН) и вычислением контекста с учетом изменяемых матриц, принадлежащих только модели. Именно тут (тут, а еще в процессе построения векторного представления из словаря токенов), с появлением объектов имитирующих память, начинается искусственый интеллект (ЛЛМ).

Проделаем вычисления для входной матрицы.

In [8]:
# По размерности входа (shape) задаем входящую размерность матриц-параметров (для корректности умножения)
# выходная размерность взята за 2 для простоты иллюстрации

d_in = inputs.shape[1]
d_out = 3

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)

# Вычисляем проекции входа на матрицы-параметры
querys = inputs @ W_query 
keys = inputs @ W_key 
values = inputs @ W_value

# Вычисляем "оценки внимания" 
attn_scores = querys @ keys.T

# Масштабирование аргумента функции на квадратный корень из размерности матрицы служит для улучшения вычислений:
# Логистическая функция напоминает ступенчатую при больших аргументах - это может привести к обнулению градиентов, 
# что уменьшает изменения матриц- параметров на шаге обучения.
d_k = keys.shape[-1]

# Вычисляем "веса внимания" 
attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)

# Вычисляем "матрицу контекста" 
context_vecs = attn_weights @ values
print('Векторное представление контекста, подсчитанное с помощую обучаемых матриц-параметров:\n',context_vecs)

Векторное представление контекста, подсчитанное с помощую обучаемых матриц-параметров:
 tensor([[0.6692, 1.0276, 1.1106],
        [0.6864, 1.0577, 1.1389],
        [0.6860, 1.0570, 1.1383],
        [0.6738, 1.0361, 1.1180],
        [0.6711, 1.0307, 1.1139],
        [0.6783, 1.0441, 1.1252]], grad_fn=<MmBackward0>)



### 3.3.4. Вычисление контекста (внимания) в одном классе

![](images/llm3.19.png) 


In [4]:
# Клас наследуется от nn.Module - фундаментального строительного блока PyTorch, содержащего механизм обратного распространения ошибки.
# (функция обратного распространения ошибки backward генерируется автоматически на основе функции forward)

import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
 
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

### 3.4 Ограничение внимания на будущие (относительно позиции) токены

Реализованная процедура вычисления внимания неправдоподобна, т.к. учитывает все токены: как до, так и после выбранной позиции. В соответствии с логикой предсказания следующего токена, следует изменить процедуру так, что бы при обработке текущего токена учитывался бы только предудущий, а не последующий контекст.  

Для этого будем маскировать веса внимания над главной диагональю матрицы весов и заново нормализовывать строки. Стоит отметить, что повторная нормализация корректна, т.е. несмотря на изначальное включение всех позиций в делитель при вычислении softmax, они не влияют на итоговый результат. 

![](images/llm3.20.png) 

In [13]:
# Реализация ограничения внимания с помощью слоя - маски. 

# Считаем веса с помощью класса

sa = SelfAttention_v1(d_in, d_out)
queries = inputs @ sa.W_query
keys = inputs @ sa.W_key 
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print('Веса внимания:\n',attn_weights)

# Создаем матрицу - маску с помощью функции tril, возращающей нижнетреугольную матрицу из матрицы единиц

context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print('Матрица - маска:\n',mask_simple)

# Применяем маску к матрицу весов
 
masked_simple = attn_weights * mask_simple
print('Умножение маски на веса:\n',masked_simple)

# Ренормализуем матрицу

row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print('Ограниченная и ренормализованная матрица весов:\n',masked_simple_norm)

Веса внимания:
 tensor([[0.1785, 0.2034, 0.2016, 0.1328, 0.1336, 0.1500],
        [0.1748, 0.2169, 0.2145, 0.1236, 0.1263, 0.1439],
        [0.1749, 0.2162, 0.2139, 0.1241, 0.1268, 0.1442],
        [0.1711, 0.1939, 0.1927, 0.1427, 0.1447, 0.1549],
        [0.1738, 0.1899, 0.1889, 0.1449, 0.1464, 0.1561],
        [0.1712, 0.2022, 0.2007, 0.1361, 0.1385, 0.1512]],
       grad_fn=<SoftmaxBackward0>)
Матрица - маска:
 tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
Умножение маски на веса:
 tensor([[0.1785, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1748, 0.2169, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1749, 0.2162, 0.2139, 0.0000, 0.0000, 0.0000],
        [0.1711, 0.1939, 0.1927, 0.1427, 0.0000, 0.0000],
        [0.1738, 0.1899, 0.1889, 0.1449, 0.1464, 0.0000],
        [0.1712, 0.2022, 0.2007, 0.1361, 0.1385, 0.1512]],
  

### 3.5 Ограничение внимания на случайные токены (прореживание, dropout) 

Прореживание (обнуление случайных значений) - распространенная практика для избежания переобучения. В ЛЛМ прореживание обычно используется в двух случаях: после вычисления оценок внимания или после вычисления матрицы контекста. 

![](images/llm3.23.png)

In [15]:
# Применяем прореживание с параметром 0.5 (выбрасывается половина значений)

torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
print(dropout(attn_weights))

tensor([[0.3570, 0.4068, 0.4032, 0.2657, 0.2672, 0.3001],
        [0.0000, 0.4338, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4277, 0.0000, 0.2536, 0.0000],
        [0.3422, 0.3877, 0.0000, 0.0000, 0.0000, 0.3097],
        [0.3476, 0.0000, 0.0000, 0.0000, 0.0000, 0.3121],
        [0.0000, 0.4045, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [ ]:
# Реализация класса с обучаемыми весами, ограничениями внимания и прореживанием 

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
           'mask',
           torch.triu(torch.ones(context_length, context_length),
           diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape # b - batch dimension, размерность блока
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = attn_weights @ values
        return context_vec

### 3.6 Распределенная обработка входных данных (multi-head attention). 

Класс - обработчик получает входные данные (преобразованный текст) и выдает векторное представление контекста. Это представление создается с помощью изменяемых в процессе обучений матриц - параметров, условно называнных "запрос", "ключ", "значение". Т.е. для каждого экземпляра класса, значения этих матриц-параметров уникальны. Метафорически можно сказать, что один экземпляр обработчика обрабатывает текст "с одной точки зрения". Продолжая метафору, скажем что "две головы лучше, чем одна". Т.е. будем обрабатывать текст с помощью множества обработчиков (каждый экземпляр обрабатывает свой кусок входного текста), увеличивая количество матриц-параметров и усложняя тем самым модель. 


![](images/llm3.25.png) 


In [ ]:
# Класс распределенной обработки получает коэфициент распределения num_heads

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
 
        self.d_out = d_out
        self.num_heads = num_heads
        # размерность выхода корректируется с учетом коэффициента 
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # так все делают
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
 
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
 
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
  
        attn_scores.masked_fill_(mask_bool, -torch.inf)
 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec


3.6 Extending single-head attention to multi-head attention

In this final section of this chapter, we are extending the previously implemented causal attention class over multiple-heads. This is also called multi-head attention.

The term "multi-head" refers to dividing the attention mechanism into multiple "heads," each operating independently. In this context, a single causal attention module can be considered single-head attention, where there is only one set of attention weights processing the input sequentially.

In the following subsections, we will tackle this expansion from causal attention to multi-head attention. The first subsection will intuitively build a multi-head attention module by stacking multiple CausalAttention modules for illustration purposes. The second subsection will then implement the same multi-head attention module in a more complicated but computationally more efficient way.
3.6.1 Stacking multiple single-head attention layers

In practical terms, implementing multi-head attention involves creating multiple instances of the self-attention mechanism (depicted earlier in Figure 3.18 in section 3.4.1), each with its own weights, and then combining their outputs. Using multiple instances of the self-attention mechanism can be computationally intensive, but it's crucial for the kind of complex pattern recognition that models like transformer-based LLMs are known for.

Figure 3.24 illustrates the structure of a multi-head attention module, which consists of multiple single-head attention modules, as previously depicted in Figure 3.18, stacked on top of each other.

Figure 3.24 The multi-head attention module in this figure depicts two single-head attention modules stacked on top of each other. So, instead of using a single matrix Wv for computing the value matrices, in a multi-head attention module with two heads, we now have two value weight matrices: Wv1 and Wv2. The same applies to the other weight matrices, Wq and Wk. We obtain two sets of context vectors Z1 and Z2 that we can combine into a single context vector matrix Z.



As mentioned before, the main idea behind multi-head attention is to run the attention mechanism multiple times (in parallel) with different, learned linear projections -- the results of multiplying the input data (like the query, key, and value vectors in attention mechanisms) by a weight matrix.

In code, we can achieve this by implementing a simple MultiHeadAttentionWrapper class that stacks multiple instances of our previously implemented CausalAttention module:

Listing 3.4 A wrapper class to implement multi-head attention

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )
 
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

For example, if we use this MultiHeadAttentionWrapper class with two attention heads (via num_heads=2) and CausalAttention output dimension d_out=2, this results in a 4-dimensional context vectors (d_out*num_heads=4), as illustrated in Figure 3.25.

Figure 3.25 Using the MultiHeadAttentionWrapper, we specified the number of attention heads (num_heads). If we set num_heads=2, as shown in this figure, we obtain a tensor with two sets of context vector matrices. In each context vector matrix, the rows represent the context vectors corresponding to the tokens, and the columns correspond to the embedding dimension specified via d_out=4. We concatenate these context vector matrices along the column dimension. Since we have 2 attention heads and an embedding dimension of 2, the final embedding dimension is 2 × 2 = 4.

To illustrate Figure 3.25 further with a concrete example, we can use the MultiHeadAttentionWrapper class similar to the CausalAttention class before:

torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
 
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

This results in the following tensor representing the context vectors:


tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],
 
        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])



The first dimension of the resulting context_vecs tensor is 2 since we have two input texts (the input texts are duplicated, which is why the context vectors are exactly the same for those). The second dimension refers to the 6 tokens in each input. The third dimension refers to the 4-dimensional embedding of each token.
Exercise 3.2 Returning 2-dimensional embedding vectors

Change the input arguments for the MultiHeadAttentionWrapper(..., num_heads=2) call such that the output context vectors are 2-dimensional instead of 4-dimensional while keeping the setting num_heads=2. Hint: You don't have to modify the class implementation; you just have to change one of the other input arguments.

In this section, we implemented a MultiHeadAttentionWrapper that combined multiple single-head attention modules. However, note that these are processed sequentially via [head(x) for head in self.heads] in the forward method. We can improve this implementation by processing the heads in parallel. One way to achieve this is by computing the outputs for all attention heads simultaneously via matrix multiplication, as we will explore in the next section.
3.6.2 Implementing multi-head attention with weight splits

In the previous section, we created a MultiHeadAttentionWrapper to implement multi-head attention by stacking multiple single-head attention modules. This was done by instantiating and combining several CausalAttention objects.

Instead of maintaining two separate classes, MultiHeadAttentionWrapper and CausalAttention, we can combine both of these concepts into a single MultiHeadAttention class. Also, in addition to just merging the MultiHeadAttentionWrapper with the CausalAttention code, we will make some other modifications to implement multi-head attention more efficiently.

In the MultiHeadAttentionWrapper, multiple heads are implemented by creating a list of CausalAttention objects (self.heads), each representing a separate attention head. The CausalAttention class independently performs the attention mechanism, and the results from each head are concatenated. In contrast, the following MultiHeadAttention class integrates the multi-head functionality within a single class. It splits the input into multiple heads by reshaping the projected query, key, and value tensors and then combines the results from these heads after computing attention.

Let's take a look at the MultiHeadAttention class before we discuss it further:

Listing 3.5 An efficient multi-head attention class

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
 
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
 
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
 
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
  
        attn_scores.masked_fill_(mask_bool, -torch.inf)
 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec



Even though the reshaping (.view) and transposing (.transpose) of tensors inside the MultiHeadAttention class looks very complicated, mathematically, the MultiHeadAttention class implements the same concept as the MultiHeadAttentionWrapper earlier.

On a big-picture level, in the previous MultiHeadAttentionWrapper, we stacked multiple single-head attention layers that we combined into a multi-head attention layer. The MultiHeadAttention class takes an integrated approach. It starts with a multi-head layer and then internally splits this layer into individual attention heads, as illustrated in Figure 3.26.

Figure 3.26 In the MultiheadAttentionWrapper class with two attention heads, we initialized two weight matrices Wq1 and Wq2 and computed two query matrices Q1 and Q2 as illustrated at the top of this figure. In the MultiheadAttention class, we initialize one larger weight matrix Wq , only perform one matrix multiplication with the inputs to obtain a query matrix Q, and then split the query matrix into Q1 and Q2 as shown at the bottom of this figure. We do the same for the keys and values, which are not shown to reduce visual clutter.



The splitting of the query, key, and value tensors, as depicted in Figure 3.26, is achieved through tensor reshaping and transposing operations using PyTorch's .view and .transpose methods. The input is first transformed (via linear layers for queries, keys, and values) and then reshaped to represent multiple heads.

The key operation is to split the d_out dimension into num_heads and head_dim, where head_dim = d_out / num_heads. This splitting is then achieved using the .view method: a tensor of dimensions (b, num_tokens, d_out) is reshaped to dimension (b, num_tokens, num_heads, head_dim).

The tensors are then transposed to bring the num_heads dimension before the num_tokens dimension, resulting in a shape of (b, num_heads, num_tokens, head_dim). This transposition is crucial for correctly aligning the queries, keys, and values across the different heads and performing batched matrix multiplications efficiently.

To illustrate this batched matrix multiplication, suppose we have the following example tensor:

a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],
 
                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

Now, we perform a batched matrix multiplication between the tensor itself and a view of the tensor where we transposed the last two dimensions, num_tokens and head_dim:

print(a @ a.transpose(2, 3))

The result is as follows:

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],
 
         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])



In this case, the matrix multiplication implementation in PyTorch handles the 4-dimensional input tensor so that the matrix multiplication is carried out between the 2 last dimensions (num_tokens, head_dim) and then repeated for the individual heads.

For instance, the above becomes a more compact way to compute the matrix multiplication for each head separately:



first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)
 
second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)


The results are exactly the same results that we obtained when using the batched matrix multiplication print(a @ a.transpose(2, 3)) earlier:

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])
 
Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])




Continuing with MultiHeadAttention, after computing the attention weights and context vectors, the context vectors from all heads are transposed back to the shape (b, num_tokens, num_heads, head_dim). These vectors are then reshaped (flattened) into the shape (b, num_tokens, d_out), effectively combining the outputs from all heads.

Additionally, we added a so-called output projection layer (self.out_proj) to MultiHeadAttention after combining the heads, which is not present in the CausalAttention class. This output projection layer is not strictly necessary (see the References section in Appendix B for more details), but it is commonly used in many LLM architectures, which is why we added it here for completeness.

Even though the MultiHeadAttention class looks more complicated than the MultiHeadAttentionWrapper due to the additional reshaping and transposition of tensors, it is more efficient. The reason is that we only need one matrix multiplication to compute the keys, for instance, keys = self.W_key(x) (the same is true for the queries and values). In the MultiHeadAttentionWrapper, we needed to repeat this matrix multiplication, which is computationally one of the most expensive steps, for each attention head.

The MultiHeadAttention class can be used similar to the SelfAttention and CausalAttention classes we implemented earlier:

torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)


s we can see based on the results, the output dimension is directly controlled by the d_out argument:

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],
 
        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])



In this section, we implemented the MultiHeadAttention class that we will use in the upcoming sections when implementing and training the LLM itself. Note that while the code is fully functional, we used relatively small embedding sizes and numbers of attention heads to keep the outputs readable.

For comparison, the smallest GPT-2 model (117 million parameters) has 12 attention heads and a context vector embedding size of 768. The largest GPT-2 model (1.5 billion parameters) has 25 attention heads and a context vector embedding size of 1600. Note that the embedding sizes of the token inputs and context embeddings are the same in GPT models (d_in = d_out).
Exercise 3.3 Initializing GPT-2 size attention modules

Using the MultiHeadAttention class, initialize a multi-head attention module that has the same number of attention heads as the smallest GPT-2 model (12 attention heads). Also ensure that you use the respective input and output embedding sizes similar to GPT-2 (768 dimensions). Note that the smallest GPT-2 model supports a context length of 1024 tokens.



3.7 Summary

    Attention mechanisms transform input elements into enhanced context vector representations that incorporate information about all inputs.
    A self-attention mechanism computes the context vector representation as a weighted sum over the inputs.
    In a simplified attention mechanism, the attention weights are computed via dot products.
    A dot product is just a concise way of multiplying two vectors element-wise and then summing the products.
    Matrix multiplications, while not strictly required, help us to implement computations more efficiently and compactly by replacing nested for-loops.
    In self-attention mechanisms that are used in LLMs, also called scaled-dot product attention, we include trainable weight matrices to compute intermediate transformations of the inputs: queries, values, and keys.
    When working with LLMs that read and generate text from left to right, we add a causal attention mask to prevent the LLM from accessing future tokens.
    Next to causal attention masks to zero out attention weights, we can also add a dropout mask to reduce overfitting in LLMs.
    The attention modules in transformer-based LLMs involve multiple instances of causal attention, which is called multi-head attention.
    We can create a multi-head attention module by stacking multiple instances of causal attention modules.
    A more efficient way of creating multi-head attention modules involves batched matrix multiplications.
